# $$Backtesting Module$$

### $$Libraries$$

In [ ]:
from typing import List, Tuple, Any
from datetime import datetime
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import gym
from gym import wrappers
import matplotlib.pyplot as plt
from pprint import pprint
pio.renderers.default = 'colab'
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### $$ Tick Processor $$

In [ ]:
def get_unix(date: str, time: str) -> int:
    format_date = "%d/%m/%Y"
    format_time = "%H:%M:%S.%f000"
    date_obj = datetime.strptime(date, format_date)
    time_obj = datetime.strptime(time, format_time)
    almost = datetime.combine(date_obj.date(), time_obj.time())
    return int(almost.timestamp())
class Tick:
    def __init__(self, line: str) -> None:
        splitted_line = line.split(',')
        self.date = splitted_line[0]
        self.time = splitted_line[1]
        self.bid = float(splitted_line[2].strip())
        self.ask = float(splitted_line[4].strip())
        self.unix_code = get_unix(splitted_line[0], splitted_line[1])

class Strategy:
    def __init__(self) -> None:
        pass
    def Von_Neuman( self, id: int, prices: List[ float ], tres_hold: float, acumulated_side_bool: bool, cant_part = 8 ) -> int:
        prices = prices[::-1]
        if ( id == 0 ) or ( acumulated_side_bool is False ) :
            up_down = [ ]
            if cant_part < len(prices):
                for index_groups in range(cant_part - 1):
                    try:
                        if prices[index_groups * 100] < prices[(index_groups + 1) * 100]:
                            up_down.append(1)
                        elif prices[index_groups * 100] > prices[(index_groups + 1) * 100]:
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try:
                    avg = sum(up_down) / len(up_down)
                    if avg > tres_hold:
                        return -1
                    else:
                        return 0
                except ZeroDivisionError:
                    return 0
            else:
                return 0
        elif ( id > 0 ) and ( acumulated_side_bool is True ) :
            up_down = []
            if cant_part < len(prices):
                for index_groups in range(cant_part - 1):
                    try:
                        if prices[index_groups * 10] < prices[(index_groups + 1) * 10]:
                            up_down.append(1)
                        elif prices[index_groups * 10] > prices[(index_groups + 1) * 10]:
                            up_down.append(-1)
                        else:
                            up_down.append(0)
                    except IndexError:
                        up_down.append(0)
                try :
                    avg = sum( up_down ) / len( up_down )
                    if -tres_hold <= avg <= tres_hold:
                        return 0
                    elif avg > 0:
                        return -1
                    else:
                        return 1
                except ZeroDivisionError:
                    return 0
            else:
                return 0

def compute_balance(orders_list: List[Any], tick: Tick) -> float:
    open_balance = 0
    for line in orders_list:
        if len(line) > 2:
            side = line[ 0 ]
            number_units = line[1]
            bid_price, ask_price = line[2], line[3]
            last_bid, last_ask = tick.bid, tick.ask
            profit_loss = 0
            if side == -1:
                profit_loss = number_units * last_bid - number_units * ask_price
            elif side == 1:
                profit_loss = number_units * bid_price - number_units * last_ask
            open_balance += profit_loss
    return open_balance * 1
def compute_drawdown(path_to_use: str, orders_list: List[Any], tick: Tick) -> float:
    if len(orders_list) > 1:
        last_balance = compute_balance(orders_list, tick)
        balances = []
        with open(path_to_use, 'r') as file_reader:
            for line in file_reader:
                balances.append(float(line.split(':')[6]))
        balances.append(last_balance)
        max_drawdown = 0
        drawdown = 0
        peak = balances[0]
        for balance in balances:
            if balance > peak:
                peak = balance
            drawdown = peak - balance
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        return max_drawdown
    else:
        return 0

def Register(path_to_use: str, line: List[Any]) -> None:
    with open(path_to_use, 'a') as file_in_process:
        for value in line:
            file_in_process.writelines(f'{value}:')
        file_in_process.writelines('\n')
def file_content(path_to_use: str) -> str:
    if os.path.exists(path_to_use):
        with open(path_to_use, 'r') as file_in_process:
            return file_in_process.read()
    else:
        return ""

def time_signal_detector( current_tick: Tick, previous_tick: Tick, pivot_unix: int ) -> bool:
    current_unix = current_tick.unix_code
    previous_unix = previous_tick.unix_code
    if current_unix <= pivot_unix:
        return True
    elif previous_unix <= pivot_unix < current_unix:
        return False

def State_detector(acumulated_side) -> bool:
    if acumulated_side >= 0 :
          return False
    else :
          return True

time_frames = [350, 357, 364, 371, 378, 385, 392, 399, 406, 413, 420, 427, 434, 441, 448,
               455, 462, 469, 476, 483, 490, 497, 504, 511, 518, 525, 532, 539, 546, 553,
               560, 567, 574, 581, 588, 595, 602, 609, 616, 623, 630, 637, 644, 651, 658,
               665, 672, 679, 686, 693]

dict_for_tres_hold = {  0.07: 'A',
                        0.08: 'B',
                        0.09: 'C',
                        0.1: 'D',
                        0.15: 'E',
                        0.16: 'F',
                        0.17: 'G',
                        0.18: 'H',
                        0.19: 'I',
                        0.2: 'J',
                        0.25: 'K' }

dict_for_decision =  {  1: 'sell',
                       -1: 'buy'  }

dict_for_time_frames = {}
for i,x in enumerate(time_frames):
    dict_for_time_frames[f'{x}'] = f'P_{i}'

def Process_day(time_frame: str, path_of_current_day: str, day: str, Strat: Strategy, sum_side_by_day = 0) -> str:
    list_for_balance = [ ]
    prices = [ ]
    index_for_balance = 0
    with open(path_of_current_day, 'r') as current_day:
        current_day_list = current_day.readlines()
        last_hour = int(((current_day_list[-2].split(','))[1].split(':'))[0]) + 1
        current_hour = 0

        # Separar la parte entera y decimal del time_frame
        int_part, *decimal_part = time_frame.split('.')
        int_part = int(int_part)
        float_part = int(decimal_part[0]) if decimal_part else 0

        # Calcular los minutos y segundos a incrementar
        increment_minutes = int_part
        increment_seconds = float_part * 6

        current_minute = 0
        current_seconds = 0

        pivot_time = f'{current_hour:02}:{current_minute:02}:{current_seconds:02}.000000000'
        id = 0

        for index in range(1, len(current_day_list)):
            current_tick = Tick(current_day_list[index])
            previous_tick = Tick(current_day_list[index - 1])
            pivot_unix = get_unix(current_tick.date, pivot_time)
            if time_signal_detector(current_tick, previous_tick, pivot_unix):
                prices.append(np.random.choice([previous_tick.bid, current_tick.ask]))
            else:
                selected_tres_hold = np.random.choice([0.07, 0.08, 0.09, 0.1, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2, 0.25])
                side = Strat.Von_Neuman(id, prices, selected_tres_hold, State_detector(sum_side_by_day))
                sum_side_by_day += side
                if side != 0:
                    list_for_balance.append([side, 1, previous_tick.bid, previous_tick.ask])
                    P_L = compute_balance(list_for_balance, previous_tick)
                    drawdown = compute_drawdown(dict_for_time_frames[time_frame]+ '_' +  day, list_for_balance, previous_tick)
                    Register(dict_for_time_frames[time_frame]+ '_' +  day, [id, dict_for_tres_hold[selected_tres_hold], dict_for_decision[side], 1, previous_tick.bid, previous_tick.ask, P_L, drawdown])
                    id += 1
                current_seconds += increment_seconds
                if current_seconds >= 60:
                    current_seconds -= 60
                    current_minute += 1

                current_minute += increment_minutes
                if current_minute >= 60:
                    extra_hours = current_minute // 60
                    current_minute = current_minute % 60
                    current_hour += extra_hours

                if current_hour >= last_hour:
                    return dict_for_time_frames[time_frame]+ '_' + day

                pivot_time = f'{current_hour:02}:{current_minute:02}:{int(current_seconds):02}.000000000'
                prices = []
    return dict_for_time_frames[time_frame]+ '_' + day

In [ ]:
folder = input("folder of Data Ticks : ")
folder_for_results = input("Write the path where u want to place the results of the backtesting : ")
archives_in_folder = os.listdir( folder )
for archive_name in archives_in_folder:
    if archive_name.endswith('.txt'):
        complete_route = os.path.join(folder, archive_name)
        for time_frame in time_frames:
            print(f'{time_frame} : {complete_route}')
            Strat = Strategy( )
            path_of_analized_day = Process_day(str(time_frame), complete_route, archive_name, Strat)
            place_path = os.path.join(folder_for_results, path_of_analized_day)
            with open(place_path, 'w') as archive:
                    archive.write(file_content(path_of_analized_day))
            if os.path.exists(path_of_analized_day):
                os.remove(path_of_analized_day)
            else:
                pass

## **Objetivo general:**

El código "Tick Processor" es un componente clave en un sistema de trading automatizado. Su función principal es procesar los datos de mercado en tiempo real (ticks) y aplicar una estrategia de trading para generar señales de compra o venta. En este caso, la estrategia implementada es la "Von Neuman", que busca identificar tendencias en los precios para tomar decisiones.

## **Partes principales del código:**

#### **Funciones auxiliares:**

- get_unix: Convierte fechas y horas en formato de tiempo Unix (segundos desde el 1 de enero de 1970), necesario para cálculos temporales.
- compute_balance, compute_drawdown: Calculan el balance actual de la cuenta y el máximo retroceso (drawdown) experimentado.
- Register, file_content: Manejan el registro de operaciones y resultados en archivos.
- time_signal_detector: Detecta si ha pasado un intervalo de tiempo específico (time frame) para aplicar la estrategia.
- State_detector: Determina si hay una posición abierta (compra o venta) en el mercado.

**Clases:**

- Tick: Representa un tick de mercado con fecha, hora, precios de compra (bid) y venta (ask).
Strategy: Contiene la lógica de la estrategia "Von Neuman", que analiza tendencias en los precios y genera señales de compra o venta.

- Lógica principal (Process_day):

Itera sobre los ticks de un día específico.
Detecta si ha pasado un intervalo de tiempo (time frame) predefinido.
Aplica la estrategia "Von Neuman" a los precios acumulados en el intervalo.
Registra las órdenes generadas (compra o venta) junto con información relevante (balance, drawdown, etc.).

### **Funcionamiento de la estrategia "Von Neuman":**

- Análisis de tendencias: Divide los precios en grupos y calcula un promedio de las variaciones entre grupos.
- Umbrales dinámicos: Selecciona aleatoriamente un umbral de sensibilidad a la tendencia (tres_hold).
- Generación de señales:
Si es el primer tick del día o no hay posición abierta, evalúa si la tendencia promedio supera el umbral positivo o negativo para generar una señal de venta o compra, respectivamente.
Si hay una posición abierta, evalúa si la tendencia promedio está dentro de un rango neutral (sin tendencia clara) para cerrar la posición, o si supera el umbral positivo o negativo para mantenerla.

### $$ Analyzer $$

In [ ]:
def compute_balance_for_big_file(orders_list: List[Any], last_bid : float,  last_ask : float ) -> float:
    open_balance = 0
    for line in orders_list:
        if len(line) > 2:
            side = line[0]
            number_units = int(line[1])
            bid_price, ask_price = float(line[2]), float(line[3])
            profit_loss = 0
            last_bid = float( last_bid )
            last_ask = float(last_ask)
            if side == 'buy' :
                profit_loss = number_units * (last_bid) - number_units * ask_price
            elif side == 'sell' :
                profit_loss = number_units * bid_price - number_units * last_ask
            open_balance += profit_loss
    return open_balance * 12500

def compute_drawdown_for_big_file( balances : List[float] ) -> float:
    if len( balances ) > 1 :
                max_drawdown = 0
                drawdown = 0
                peak = balances[0]
                for balance in balances :
                    if balance > peak :
                        peak = balance
                    drawdown = peak - balance
                    if drawdown > max_drawdown:
                        max_drawdown = drawdown
                return max_drawdown
    else:
        return 0
def Register(path_to_use: str, line: List[Any]) -> None:
    with open(path_to_use, 'a') as file_in_process:
        for value in line:
            file_in_process.writelines(f'{value}:')
        file_in_process.writelines('\n')
def file_content(path_to_use: str) -> str:
    if os.path.exists(path_to_use):
        with open(path_to_use, 'r') as file_in_process:
            return file_in_process.read()
    else:
        return ""
def chronological_organizer(Profile, folder) :
  Profile = f'P_{Profile}'
  archives_in_folder = os.listdir( folder )####
  df = { 'Archive':[ ], 'Month':[ ],'Year':[],'Day':[] }
  for archive in archives_in_folder :
      if len(archive)== 19:
         pr = archive[:4]
      else :
         pr = archive[:3]
      if archive.endswith('.txt') and pr == Profile:
          rout = os.path.join( folder, archive )
          with open( rout, 'r' ) as reading_analysis :
              lines = reading_analysis.readlines( )
              if len(lines) > 0:
                    df['Archive'].append(archive)
                    if len(archive) == 19 :
                        df['Day'].append('P_1_01_03_2023.txt'[4:6])
                        df['Month'].append('P_1_01_03_2023.txt'[7:9])
                        df['Year'].append('P_1_01_03_2023.txt'[10:14])
                    else :
                        df['Day'].append('P_1_01_03_2023.txt'[5:7])
                        df['Month'].append('P_1_01_03_2023.txt'[8:10])
                        df['Year'].append('P_1_01_03_2023.txt'[11:15])
  df= pd.DataFrame(df)
  df= df.sort_values( by= ['Year','Month','Day'])
  return df.Archive
def merge_txt_files(chronological_order, profile, folder) :
  content = ""
  path_of_analized_day = f'Profile_{profile}_sum.txt'
  list_for_balance = [ ]
  list_for_drawdown = [ ]
  with open(path_of_analized_day, 'a') as file_in_process:
    for archive_name in chronological_order :
        if archive_name.endswith('.txt'):
            complete_route = os.path.join( folder, archive_name )
            with open( complete_route, 'r' ) as current_day:
                lines = current_day.readlines( )
                if len( lines ) > 0 :
                    for order in lines :
                        splitted = order.split( ':' )
                        list_for_balance.append( splitted[ 2: 6 ] )
                        P_L = compute_balance_for_big_file(list_for_balance, splitted[4], splitted[5] )
                        list_for_drawdown.append(P_L)
                        drawdown = compute_drawdown_for_big_file( list_for_drawdown )
                        #file_in_process.writelines(f'{splitted[0]}:{splitted[1]}:{splitted[2]}:{splitted[3]}:{splitted[4]}:{splitted[5]}:{P_L}:{drawdown}\n')
                        content +=f'{splitted[0]}:{splitted[1]}:{splitted[2]}:{splitted[3]}:{splitted[4]}:{splitted[5]}:{P_L}:{drawdown}\n'
  return content
      #place_path = os.path.join("C:\\Users\\lorenzo1\\Desktop\\trading\\Summary_folder", path_of_analized_day)
      #with open(place_path, 'w') as archive:
      #             archive.write(file_content(path_of_analized_day))

## **Objetivo general:**

Este código se enfoca en el procesamiento de grandes cantidades de datos de trading para calcular métricas clave como el balance de la cuenta y el máximo retroceso (drawdown) a lo largo del tiempo. Esto es esencial para evaluar el rendimiento de una estrategia de trading a gran escala y para tomar decisiones informadas sobre la gestión del riesgo.

## **Partes principales del código:**

### **Funciones auxiliares:**

- compute_balance_for_big_file: Calcula el balance de la cuenta a partir de una lista de órdenes (compras y ventas) y los precios de compra (bid) y venta (ask) más recientes. A diferencia de la versión anterior (compute_balance), esta función está diseñada para manejar grandes cantidades de datos y multiplica el balance por un factor (12500) que podría representar el tamaño del contrato.
- compute_drawdown_for_big_file: Calcula el máximo retroceso (drawdown) a partir de una lista de balances históricos. El drawdown es la pérdida máxima desde un pico anterior en el balance, y es un indicador importante del riesgo de una estrategia.
- Register: Registra líneas de datos (posiblemente órdenes u otras métricas) en un archivo, agregando una nueva línea al final.
- file_content: Lee el contenido completo de un archivo de texto.
chronological_organizer: Organiza los archivos de datos de trading en orden cronológico (por fecha), asegurando que el análisis se realice en el orden correcto de las operaciones.
- Lógica principal (merge_txt_files):
Itera sobre los archivos de datos de trading en orden cronológico.
Lee las líneas de cada archivo, que representan las órdenes de compra o venta.
Calcula el balance y el drawdown para cada orden, utilizando las funciones auxiliares mencionadas.
Acumula los datos en listas para su posterior análisis o registro.

### **Funcionamiento del cálculo de balance y drawdown:**

##### **compute_balance_for_big_file:**

- Itera sobre cada orden en la lista.
- Determina si la orden es de compra ("buy") o venta ("sell").
- Calcula la ganancia o pérdida de la orden multiplicando la diferencia entre el precio de ejecución y el precio actual (bid o ask) por el número de unidades operadas.
- Acumula las ganancias y pérdidas en la variable open_balance.
- Multiplica el balance final por un factor (12500) para ajustarlo al tamaño del contrato.

#### **compute_drawdown_for_big_file:**

- Itera sobre la lista de balances históricos.
- Rastrea el pico máximo alcanzado en el balance hasta el momento.
- Calcula el retroceso (drawdown) como la diferencia entre el pico actual y el balance actual.
- Actualiza el máximo retroceso si se encuentra un nuevo valor más alto.

In [ ]:
folder_for_summaries = input("Write the path where u want to place the summary by profile : ")
for i in range(99):
    path_of_analized_day = f'Profile_{i}_sum.txt'
    order = chronological_organizer( i,folder_for_results )
    content = merge_txt_files( order, i, folder_for_results )
    with open(os.path.join(folder_for_summaries,path_of_analized_day),'w') as writing_archive :
      writing_archive.write(content)


    '''
    with open(os.path.join("C:\\Users\\lorenzo1\\Desktop\\trading\\Summary_folder", path_of_analized_day), 'w') as archive:
       if content is not None :
            archive.write(content)
    '''

## **Objetivo general:**

Esta sección de código se encarga de resumir los resultados de trading para cada perfil individual. Recopila la información de las operaciones realizadas (contenida en archivos separados por día y perfil) y la consolida en un único archivo de resumen por perfil. Esto facilita el análisis global del rendimiento de cada estrategia asociada a un perfil.

### **Lógica principal:**

**Organización cronológica:** Para cada perfil:
- Llama a la función chronological_organizer para obtener una lista ordenada cronológicamente de los archivos de resultados diarios de ese perfil.
- Construye el nombre del archivo de resumen (Profile_{i}_sum.txt, donde i es el número de perfil).

**Consolidación de resultados:** Llama a la función merge_txt_files para:

- Leer los archivos de resultados diarios en orden cronológico.
- Consolidar la información relevante de cada operación (ID, acción, cantidad, precios, balance, drawdown) en una cadena de texto (content).
- Calcular el balance acumulado y el drawdown máximo para cada operación.

Ejemplo de línea en un archivo de resumen:

0:sell:1:1.23456:1.23457:-125.00:125.00

**Donde:**

- 0: ID de la operación
- sell: Acción realizada (compra o venta)
- 1: Cantidad operada
- 1.23456: Precio de compra (bid)
- 1.23457: Precio de venta (ask)
- -125.00: Balance de la operación
- 125.00: Drawdown máximo hasta ese momento

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



'P_7_01-07-2022.txt'

In [ ]:
###############################
drive.mount( '/content/drive' )#######################
folder = '/content/drive/My Drive/for_drive/summary' #
archives_in_folder = os.listdir( folder )#############

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


$$ \text{ Drawdown and Balance over Time
}$$

In [ ]:
folder_for_summaries='/content/drive/My Drive/for_drive/summary'

In [ ]:
df = {'Balance':[],'Drawdown':[]}
with open (os.path.join(folder_for_summaries,'Profile_0_sum.txt')) as reading_file :
  lines = reading_file.readlines()
  for line in lines :
    splitted = line.split(':')
    df['Balance'].append(float(splitted[6]))
    df['Drawdown'].append(float(splitted[7]))

df = pd.DataFrame(df)
X= [i for i in range(len(df['Balance']))]
fig1 = px.bar(df, x=X, y='Balance', title='Balance over Time',color='Balance')
fig1.update_layout(xaxis_title='Time', yaxis_title='Balance')

# Crear el segundo gráfico de línea para 'Drawdown'
fig2 = px.bar(df, x=X, y='Drawdown', title='Drawdown over Time',color='Drawdown')
fig2.update_layout(xaxis_title='Time', yaxis_title='Drawdown')

# Mostrar los gráficos
fig1.show()
fig2.show()



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



### Primer grafica

**Período Inicial (0 a 100):**

- Observación: El saldo se mantiene cercano al nivel cero con pequeñas fluctuaciones.
- Interpretación: Durante este intervalo, el algoritmo opera principalmente en un balance bajo. Las pequeñas variaciones sugieren que las decisiones de compra y venta no generan cambios significativos en el saldo, indicando posiblemente una baja volatilidad o una ineficacia en las estrategias a corto plazo.

**Primer Descenso Notable (100 a 300):**

- Observación: El saldo muestra una tendencia negativa, cayendo hasta alrededor de -20k.
- Interpretación: En esta fase, el algoritmo sigue operando en un balance bajo, pero con resultados negativos acumulados. Esto puede sugerir que las condiciones de mercado adversas afectan negativamente las operaciones a corto plazo o que las estrategias de corto plazo necesitan ajustes.

**Estabilización y Nueva Caída (300 a 500):**

- Observación: Tras una breve estabilización, el saldo continúa descendiendo hasta cerca de -40k.
- Interpretación: La operación sigue predominantemente en un balance bajo. La estabilización breve indica una pausa en las pérdidas, pero la posterior caída sugiere que el algoritmo no logra adaptarse adecuadamente a las condiciones de mercado en este período.

**Recuperación Significativa (500 a 700):**

- Observación: A partir del punto 500, el saldo se recupera de forma significativa, alcanzando hasta 80k.
- Interpretación: Esta recuperación está asociada con un cambio a operaciones en un balance alto. La estrategia parece ser más efectiva en estos periodos más largos, lo que resulta en ganancias rápidas y sostenidas. Este cambio de perfil sugiere una mejor adaptación del algoritmo a las condiciones de mercado a largo plazo.

**Período de Alta Volatilidad (600 a 700):**

- Observación: Grandes oscilaciones en el saldo, con variaciones de colores entre amarillo y naranja.
- Interpretación: Este período indica alta volatilidad en los resultados, reflejando un mercado altamente fluctuante o un trading agresivo en el balance alto. El algoritmo parece estar ajustando sus decisiones en respuesta a rápidas variaciones del mercado, resultando en significativas fluctuaciones del saldo.

**Corrección Final y Estabilidad (700 en adelante):**

- Observación: El saldo desciende y se estabiliza alrededor del nivel cero.
- Interpretación: Hacia el final del gráfico, el saldo se estabiliza nuevamente, pero con pequeñas oscilaciones negativas. Esto puede ser una fase de corrección después de la alta volatilidad previa, donde el algoritmo ajusta sus estrategias posiblemente regresando a balance bajo o adoptando una postura más conservadora.

### Segunda grafica

**Período Inicial (0 a 100):**

- Observación: El drawdown se mantiene cerca de cero, indicando muy pocas pérdidas.
- Interpretación: En este intervalo, el algoritmo opera principalmente en un balance bajo. La estabilidad cercana a cero sugiere que las pérdidas son mínimas, posiblemente debido a condiciones de mercado favorables o una estrategia eficaz a corto plazo.

**Incremento en el Drawdown (100 a 300):**

- Observación: El drawdown empieza a aumentar gradualmente, alcanzando aproximadamente 20k.
- Interpretación: Esta fase indica un incremento en las pérdidas acumuladas. El algoritmo sigue operando en un balance bajo, pero con resultados negativos. Esto puede reflejar una estrategia menos efectiva en estas condiciones de mercado o un aumento en la volatilidad del mercado que impacta negativamente las operaciones a corto plazo.

**Aumento Continuo del Drawdown (300 a 500):**

- Observación: El drawdown continúa incrementándose de manera más pronunciada, llegando hasta aproximadamente 40k.
- Interpretación: La operación sigue en un balance bajo, con un continuo incremento en las pérdidas acumuladas. Esto sugiere que el algoritmo no está adaptándose adecuadamente a las condiciones de mercado en este período, resultando en un drawdown significativo.

**Drawdown Máximo y Recuperación Parcial (500 a 700):**

- Observación: El drawdown alcanza niveles máximos, superando los 80k.
- Interpretación: Este período muestra un cambio a operaciones con un balance alto, aunque con un drawdown elevado. Las pérdidas acumuladas son significativas, indicando que, a pesar de cambiar la estrategia a largo plazo, el algoritmo sigue experimentando pérdidas considerables.

**Período de Alta Volatilidad en Drawdown (600 a 700):**

- Observación: El drawdown se estabiliza pero en un nivel alto, con fluctuaciones menores.
- Interpretación: Aunque el algoritmo sigue operando con un balance alto, hay fluctuaciones menores que indican periodos de recuperación parcial seguidos de nuevas caídas. Esto puede reflejar condiciones de mercado volátiles donde las estrategias de largo plazo aún están enfrentando dificultades para recuperar el saldo perdido.

**Reducción y Estabilización del Drawdown (700 en adelante):**

- Observación: El drawdown empieza a reducirse y se estabiliza en un nivel más bajo.
- Interpretación: Hacia el final del gráfico, el algoritmo parece ajustar sus estrategias, logrando reducir el drawdown. Esto sugiere una mejora en la efectividad del algoritmo, posiblemente debido a una combinación de ajustes en la estrategia y condiciones de mercado más favorables.

In [ ]:
df = {'Profile':[],'Drawdown':[],'Balance':[]}
for archive in archives_in_folder :
  if archive.endswith('.txt') :
    complete_route = os.path.join(folder,archive)
    with open(complete_route, 'r') as reading :
      all_list = reading.readlines()
      if len(all_list) > 0 :
        last_line_splitted = all_list[-1].split(':')
        if len(archive)==17 :
          profile = archive[8:9]
        else :
          profile = archive[8:10]
        df['Profile'].append(int(profile))
        df['Balance'].append(float(last_line_splitted[6]))
        df['Drawdown'].append(float(last_line_splitted[7]))
      else :
        print(all_list)
df = pd.DataFrame(df)
df = df.sort_values(by="Profile")
df.head(85)


,Profile,Drawdown,Balance
4,0,92797.50,-11067.50
6,1,57595.00,-11545.00
3,2,54331.25,-15175.00
0,3,40846.25,-16883.75
7,4,40782.50,-11286.25
...,...,...,...
93,80,4275.00,1885.00
98,81,448.75,-450.00
96,82,1328.75,101.25
86,83,2166.25,2942.50


$$
\text{(Drawdown vs Balance) by Profile}
$$


In [ ]:
fig = px.scatter(df,
                 x="Balance", y="Drawdown",
                 color="Profile",
                 hover_name="Profile"
                 )
fig.update_layout(title='Drawdown x Balance x Profile')
highlighted_point = {
    "x": [sum(df['Balance']) / len(df['Balance'])],  # Ejemplo de coordenada x
    "y": [sum(df['Drawdown']) / len(df['Drawdown'])],  # Ejemplo de coordenada y
    "text": ["Punto Destacado"],  # Texto para el hover
    "marker": {"color": "red", "size": 20, "symbol": "star"}
}
fig.add_trace(
    go.Scatter(
        x=highlighted_point["x"],
        y=highlighted_point["y"],
        mode='markers+text',
        text=highlighted_point["text"],
        textposition='top center',
        marker=highlighted_point["marker"],
        name='Punto Destacado'
    )
)
X= sum( df['Balance'] ) / len( df['Balance'] )
Y= sum( df['Drawdown'] ) / len(df['Drawdown'])
print(f'centroide X:{X}, Y:{Y}')
fig.show()


centroide X:1316.95707070706, Y:10535.239898989897


## Drawdown vs Balance vs Perfil

**Distribución General:**

- Observación: Los perfiles se distribuyen a lo largo del eje del balance desde aproximadamente -15k hasta 10k y en el eje de drawdown desde 0 hasta más de 80k.
- Interpretación: Esta dispersión indica que el algoritmo experimenta una amplia gama de resultados en términos de balance y drawdown, dependiendo del timeframe utilizado.

**Timeframes Cortos (Perfiles Morados/Azules):**

- Observación: Los perfiles con colores más morados/azules (timeframes cortos) tienden a agruparse en regiones con balances negativos y drawdowns elevados.
- Interpretación: Los timeframes cortos parecen estar asociados con mayores drawdowns y balances negativos, sugiriendo que la estrategia del algoritmo es menos efectiva en periodos de tiempo más cortos, posiblemente debido a la mayor volatilidad o a la ineficacia de las señales de trading en estos intervalos.

**Timeframes Largos (Perfiles Amarillos/Naranjas):**

- Observación: Los perfiles con colores más amarillos/naranjas (timeframes largos) se agrupan más cerca del eje de balance positivo y presentan drawdowns más controlados.
- Interpretación: Los timeframes más largos parecen ser más beneficiosos para el algoritmo, mostrando menores drawdowns y balances más positivos. Esto indica una mayor estabilidad y efectividad de la estrategia en periodos de tiempo más largos, donde las tendencias de mercado pueden ser más fáciles de captar y aprovechar.

**Centroide (Punto Destacado):**

- Observación: El punto destacado, que representa el centroide de todos los perfiles, se sitúa cerca del eje de balance cero y en una región de drawdown relativamente bajo.
- Interpretación: El centroide indica que, en promedio, el algoritmo tiende a operar con un balance cercano a cero y un drawdown moderado. Esto sugiere que, considerando todos los timeframes, la estrategia del algoritmo tiende a ser neutral en términos de ganancia/pérdida neta, con una exposición al riesgo moderada.

### **Conclusiones:**

**Estrategia de Timeframes Cortos:**

- Los timeframes cortos (perfiles morados/azules) tienden a estar asociados con resultados negativos y mayores drawdowns. Esto sugiere que las decisiones de trading basadas en intervalos cortos de tiempo podrían estar expuestas a una mayor volatilidad y a señales menos fiables, resultando en mayores pérdidas acumuladas.

**Estrategia de Timeframes Largos:**

- Los timeframes largos (perfiles amarillos/naranjas) muestran una tendencia hacia balances positivos y drawdowns más controlados. Esto indica que la estrategia del algoritmo es más efectiva en estos intervalos, probablemente debido a una mejor capacidad para identificar y seguir las tendencias de mercado más estables.